In [1]:
import os
import subprocess
import h5py
import matplotlib.pyplot as plt
import string
import numpy as np
from glob import glob
from skimage.io import imread
from skimage.transform import downscale_local_mean
import pandas as pd
import seaborn as sns
from pywavesurfer import ws
from scipy.io import savemat,loadmat
import numpy as np
# From Process H5 file (duplicate modules deleted)
from pywavesurfer import ws

- Filter data for ploting (date, curated well, dye, etc)

In [4]:
sumPath = r'Y:\GECI_screening\wHaloCaMP'
dates = ['20230515_wHaloCaMP_raw','20230522_wHaloCaMP_raw']
df = pd.DataFrame()
for date in dates:
    data = glob(os.path.join(sumPath,'combo & single mutants 2023',date)+'\\*.pkl')[0]
    df_temp = pd.read_pickle(data)
    df = pd.concat([df,df_temp],axis=0)

In [5]:
# df.shape
# df.head()
# df
# df.Construct.unique().shape
# df.Construct.unique()
# df.sort_values(by='Construct',ascending=True).Construct.unique()

# Following code is checking if 139.1 and 141.1 ever co-existed in one plate
df_test = df[df['Construct'].isin(['139dot1','141dot1'])]
for p in df_test['Plate'].unique():
    df_p = df_test[df_test['Plate']==p]
    if df_p['Construct'].unique().shape[0]!=1:
        print(df_p['Construct'].unique())

#### Output wHaloCaMP_norm.csv

In [30]:
stim = '160FP'
df_sel = df[(df['Curated']=='False')&(df['Stim']==stim)]

In [31]:
print(df_sel.shape)
df_sel = df_sel.dropna()
df_sel.shape

(1660, 20)


(1520, 20)

In [32]:
df_plot = pd.DataFrame()
for plate in df_sel.Plate.unique():
    df_ctr = df_sel[(df_sel['Plate']==plate)&(df_sel['Construct']=='139dot1')&(df_sel['Stim']==stim)] 
    if len(df_ctr) == 0:
        df_ctr = df_sel[(df_sel['Plate']==plate)&(df_sel['Construct']=='141dot1')&(df_sel['Stim']==stim)]

    ctr_dff = df_ctr.dFF_peak.mean()
    ctr_f0 = df_ctr.F0.mean()
    ctr_rise = df_ctr.Half_rise.mean()
    ctr_decay = df_ctr.Half_decay.mean()
    
    ctr_dff_sem = df_ctr.dFF_peak.sem()
    ctr_f0_sem = df_ctr.F0.sem()
    ctr_rise_sem = df_ctr.Half_rise.sem()
    ctr_decay_sem = df_ctr.Half_decay.sem()
    
    for c in df_sel[(df_sel['Plate']==plate)&(df_sel['Construct']!='139dot1')&(df_sel['Construct']!='141dot1')].Construct.unique():
        df_temp = pd.DataFrame()
        df_c = df_sel[(df_sel['Plate']==plate)&(df_sel['Construct']==c)]
        norm_dff = df_c.dFF_peak/ctr_dff
        norm_f0 = df_c.F0/ctr_f0
        norm_rise = df_c.Half_rise/ctr_rise
        norm_decay = df_c.Half_decay/ctr_decay

        df_temp['unnorm_dff'] = df_c.dFF_peak
        df_temp['unnorm_f0'] = df_c.F0
        df_temp['unnorm_rise'] = df_c.Half_rise
        df_temp['unnorm_decay'] = df_c.Half_decay
        
        df_temp['ctr_dff'] = ctr_dff
        df_temp['ctr_f0'] = ctr_f0
        df_temp['ctr_rise'] = ctr_rise
        df_temp['ctr_decay'] = ctr_decay
        
        df_temp['ctr_dff_sem'] = ctr_dff_sem
        df_temp['ctr_f0_sem'] = ctr_f0_sem
        df_temp['ctr_rise_sem'] = ctr_rise_sem
        df_temp['ctr_decay_sem'] = ctr_decay_sem
        
        df_temp['norm_dff'] = norm_dff
        df_temp['norm_f0'] = norm_f0
        df_temp['norm_rise'] = norm_rise
        df_temp['norm_decay'] = norm_decay
        
        df_temp['construct'] = c
        df_temp['Date'] = df_c.Date.unique()[0]
        df_temp['Plate'] = plate
        df_temp['Stim'] = stim
               
        df_plot = pd.concat([df_plot,df_temp],axis=0,ignore_index=True)

In [33]:
df_plot.shape
# df_plot.dropna().shape
# df_ctr

(1348, 20)

In [34]:
from scipy.stats import ttest_1samp

df_plot_mean = pd.DataFrame()

Construct = []
Plate = []
Date = []

norm_dFF = []
norm_F0 = []
norm_Rise = []
norm_Decay = []

norm_dFF_sem = []
norm_F0_sem = []
norm_Rise_sem = []
norm_Decay_sem = []

norm_dFF_P = []
norm_F0_P = []
norm_Rise_P = []
norm_Decay_P = []

unnorm_dFF = []
unnorm_F0 = []
unnorm_Rise = []
unnorm_Decay = []

unnorm_dFF_sem = []
unnorm_F0_sem = []
unnorm_Rise_sem = []
unnorm_Decay_sem = []

ctr_dFF = []
ctr_F0 = []
ctr_Rise = []
ctr_Decay = []

ctr_dFF_sem = []
ctr_F0_sem = []
ctr_Rise_sem = []
ctr_Decay_sem = []

N = []

for p in df_plot.Plate.unique():
    df_plot_t = df_plot[df_plot['Plate']==p]
    d = df_plot_t.Date.unique()[0]
    for c in df_plot_t.construct.unique():
        df_c = df_plot_t[df_plot_t['construct']==c]

        unnorm_dff = df_c.unnorm_dff.mean()
        unnorm_f0 = df_c.unnorm_f0.mean()
        unnorm_rise = df_c.unnorm_rise.mean()
        unnorm_decay = df_c.unnorm_decay.mean()

        unnorm_dff_sem = df_c.unnorm_dff.sem()
        unnorm_f0_sem = df_c.unnorm_f0.sem()
        unnorm_rise_sem = df_c.unnorm_rise.sem()
        unnorm_decay_sem = df_c.unnorm_decay.sem()

        if len(df_c.ctr_dff.unique())!=1:
            print(plate,c)
        ctr_dff = df_c.ctr_dff.unique()[0]
        ctr_f0 = df_c.ctr_f0.unique()[0]
        ctr_rise = df_c.ctr_rise.unique()[0]
        ctr_decay = df_c.ctr_decay.unique()[0]

        ctr_dff_sem = df_c.ctr_dff_sem.unique()[0]
        ctr_f0_sem = df_c.ctr_f0_sem.unique()[0]
        ctr_rise_sem = df_c.ctr_rise_sem.unique()[0]
        ctr_decay_sem = df_c.ctr_decay_sem.unique()[0]

        norm_dff = df_c.norm_dff.mean()
        norm_f0 = df_c.norm_f0.mean()
        norm_rise = df_c.norm_rise.mean()
        norm_decay = df_c.norm_decay.mean()

        norm_dff_sem = df_c.norm_dff.sem()
        norm_f0_sem = df_c.norm_f0.sem()
        norm_rise_sem = df_c.norm_rise.sem()
        norm_decay_sem = df_c.norm_decay.sem()

        Construct.append(c)    
        norm_dFF.append(norm_dff)
        norm_F0.append(norm_f0)
        norm_Rise.append(norm_rise)
        norm_Decay.append(norm_decay)

        norm_dFF_sem.append(norm_dff_sem)
        norm_F0_sem.append(norm_f0_sem)
        norm_Rise_sem.append(norm_rise_sem)
        norm_Decay_sem.append(norm_decay_sem)

        norm_dFF_P.append(ttest_1samp(df_c.norm_dff, 1, axis=0, alternative='greater')[1])
        norm_F0_P.append(ttest_1samp(df_c.norm_f0, 1, axis=0, alternative='greater')[1])
        norm_Rise_P.append(ttest_1samp(df_c.norm_rise, 1, axis=0, alternative='less')[1])
        norm_Decay_P.append(ttest_1samp(df_c.norm_decay, 1, axis=0, alternative='less')[1])

        unnorm_dFF.append(unnorm_dff)
        unnorm_F0.append(unnorm_f0)
        unnorm_Rise.append(unnorm_rise)
        unnorm_Decay.append(unnorm_decay)

        unnorm_dFF_sem.append(unnorm_dff_sem)
        unnorm_F0_sem.append(unnorm_f0_sem)
        unnorm_Rise_sem.append(unnorm_rise_sem)
        unnorm_Decay_sem.append(unnorm_decay_sem)

        ctr_dFF.append(ctr_dff)
        ctr_F0.append(ctr_f0)
        ctr_Rise.append(ctr_rise)
        ctr_Decay.append(ctr_decay)

        ctr_dFF_sem.append(ctr_dff_sem)
        ctr_F0_sem.append(ctr_f0_sem)
        ctr_Rise_sem.append(ctr_rise_sem)
        ctr_Decay_sem.append(ctr_decay_sem)

        N.append(len(df_c))
        Plate.append(p)
        Date.append(d)
    
df_plot_mean['Construct'] = Construct
df_plot_mean['Date'] = Date
df_plot_mean['Plate'] = Plate
df_plot_mean['Stim'] = stim

df_plot_mean['Norm \u0394F/F0'] = norm_dFF
df_plot_mean['Norm \u0394F/F0 sem'] = norm_dFF_sem
df_plot_mean['P val (Norm \u0394F/F0)'] = norm_dFF_P

df_plot_mean['Norm F0'] = norm_F0
df_plot_mean['Norm F0 sem'] = norm_F0_sem
df_plot_mean['P val (Norm F0)'] = norm_F0_P

df_plot_mean['Norm half rise time'] = norm_Rise
df_plot_mean['Norm half rise time sem'] = norm_Rise_sem
df_plot_mean['P val (Norm half rise time)'] = norm_Rise_P

df_plot_mean['Norm half decay time'] = norm_Decay
df_plot_mean['Norm half decay time sem'] = norm_Decay_sem
df_plot_mean['P val (Norm half decay time)'] = norm_Decay_P

df_plot_mean['Unnorm \u0394F/F0'] = unnorm_dFF
df_plot_mean['Unnorm \u0394F/F0 sem'] = unnorm_dFF_sem

df_plot_mean['Unnorm F0'] = unnorm_F0
df_plot_mean['Unnorm F0 sem'] = unnorm_F0_sem

df_plot_mean['Unnorm half rise time'] = unnorm_Rise
df_plot_mean['Unnorm half rise time sem'] = unnorm_Rise_sem

df_plot_mean['Unnorm half decay time'] = unnorm_Decay
df_plot_mean['Unnorm half decay time sem'] = unnorm_Decay_sem


df_plot_mean['Ctr \u0394F/F0'] = ctr_dFF
df_plot_mean['Ctr \u0394F/F0 sem'] = ctr_dFF_sem

df_plot_mean['Ctr F0'] = ctr_F0
df_plot_mean['Ctr F0 sem'] = ctr_F0_sem

df_plot_mean['Ctr half rise time'] = ctr_Rise
df_plot_mean['Ctr half rise time sem'] = ctr_Rise_sem

df_plot_mean['Ctr half decay time'] = ctr_Decay
df_plot_mean['Ctr half decay time sem'] = ctr_Decay_sem

df_plot_mean['N'] = N

C:\Users\zhengj10\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3723: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\zhengj10\Anaconda3\lib\site-packages\numpy\core\_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [35]:
# df_ben = pd.DataFrame() # Run only when processing 001FP data so following stimuli data don't overwrite

In [36]:
print(df_ben.shape)
df_ben = pd.concat([df_ben,df_plot_mean], axis=0, ignore_index=True)
df_ben.shape

(1110, 33)


(1480, 33)

In [37]:
df_ben.dropna().shape

(1468, 33)

In [41]:
df_csv = pd.read_csv(r'Z:\GECIScreenData\GECI_Imaging_Data\wHaloCaMP\summary\wHaloCaMP_norm.csv')
print(df_csv.shape)
df_ben.columns==df_csv.columns # Need to manually delete unnamed column of df_csv

(17588, 33)


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [40]:
# df_ben.columns
# df_csv.columns

Index(['Construct', 'Date', 'Plate', 'Stim', 'Norm ΔF/F0', 'Norm ΔF/F0 sem',
       'P val (Norm ΔF/F0)', 'Norm F0', 'Norm F0 sem', 'P val (Norm F0)',
       'Norm half rise time', 'Norm half rise time sem',
       'P val (Norm half rise time)', 'Norm half decay time',
       'Norm half decay time sem', 'P val (Norm half decay time)',
       'Unnorm ΔF/F0', 'Unnorm ΔF/F0 sem', 'Unnorm F0', 'Unnorm F0 sem',
       'Unnorm half rise time', 'Unnorm half rise time sem',
       'Unnorm half decay time', 'Unnorm half decay time sem', 'Ctr ΔF/F0',
       'Ctr ΔF/F0 sem', 'Ctr F0', 'Ctr F0 sem', 'Ctr half rise time',
       'Ctr half rise time sem', 'Ctr half decay time',
       'Ctr half decay time sem', 'N'],
      dtype='object')

In [42]:
df_ben = pd.concat([df_ben,df_csv],axis=0,ignore_index=True)
df_ben.shape

(19068, 33)

In [43]:
print(df_ben.shape) # NA value due to statistics based on low n
df_ben = df_ben.dropna()
df_ben = df_ben.sort_values(by=['Construct', 'Stim'],ascending = [True, True])
df_ben.index = np.arange(len(df_ben))
df_ben.shape

(19068, 33)


(19056, 33)

In [ ]:
df_ben.to_csv(r'Z:\GECIScreenData\GECI_Imaging_Data\wHaloCaMP\summary\wHaloCaMP_norm.csv')

In [ ]:
df_ben['Date'].unique()